# ベイズ勉強会 Part 3 1次元ガウス分布のベイズ推論
> 1次元ガウス分布のベイズ推論を実践する

- toc: true 
- badges: false
- comments: true
- categories: [bayes]
- image: images/dag1.png

参考図書 {% fn 1 %}

# 1次元ガウス分布

1次元のガウス分布(以下本ページでは単に「ガウス分布」と呼ぶ)は、次の確率密度関数で表される$x \in \mathbb{R}$を生成する確率分布である。

> Important: 1次元ガウス分布の確率密度関数$$\mathcal{N}(x|\mu,\sigma^2) = \frac{1}{\sqrt{2 \pi \sigma^2}} exp\{-\frac{(x-\mu)^2}{2\sigma^2}\}$$

パラメータは$\mu \in \mathbb{R}, \sigma^2 \in \mathbb{R^+}$の2つで、それぞれ平均と分散である。精度パラメータ$\lambda = \sigma^{-2}$を用いて書くこともある。精度で書くと以下のようになる。

> Important: 精度で表した1次元ガウス分布の確率密度関数$$\mathcal{N}(x|\mu,\lambda^{-1}) = \frac{1}{\sqrt{2 \pi}} \lambda^{\frac{1}{2}}exp\{-\frac{1}{2}(x-\mu)^2 \lambda\}$$

以下、観測されたデータ$\mathcal{D}=\{x_1,\dots,x_N\}$の各点$x_n$と未知の観測$x_*$は同じ1次元ガウス分布から独立に生成されたと仮定してベイズ推論を行う。

# 平均が未知、精度が既知の場合

ガウス分布の2つのパラメータのうち、精度パラメータ$\lambda$が既知である状況でベイズ推論を行う。

## 同時分布の構築

平均$\mu$だけが未知という条件で、尤度関数をガウス分布にした場合、そのパラメータ$\mu$の事前分布はどうすればよいだろうか。$\mu$の条件は$\mu \in \mathbb{R}$であることのみであり、実数を1つ出力する分布を事前分布とすればベイズ推論ができそうだ。このような分布は様々だが、1次元ガウス分布を用いることで事後分布も1次元ガウス分布となることが知られている。つまり尤度関数が1次元ガウス分布の場合の共役事前分布は1次元ガウス分布である。これを用いて同時分布を構築すると以下のようになる。

$$
\begin{eqnarray}
    p(\mathcal{D},x_*,\mu) &=& p(\mathcal{D}|\mu)p(x_*|\mu)p(\mu) \\
    p(\mathcal{D}|\mu) &=& \Pi_{n=1}^{N} \mathcal{N}(x_n|\mu, \lambda^{-1}) \\
    p(x_*|\mu) &=& \mathcal{N}(x_*|\mu, \lambda^{-1}) \\
    p(\mu) &=& \mathcal{N}(\mu|m, \lambda_{\mu}^{-1})
\end{eqnarray}
$$

$p(\mu)$の$m \in \mathbb{R}, \lambda_{\mu} \in \mathbb{R^+}$は固定されたハイパーパラメータである。

## 事後分布の推論

事後分布$p(\mu|\mathcal{D})$を求める。ベイズの定理から次のように書ける。分母は形状には関わらないので省く。

$$
\begin{eqnarray}
    p(\mu|\mathcal{D}) &\propto& p(\mathcal{D}|\mu) p(\mu) \\
    &=& \{\Pi_{n=1}^{N} p(x_n|\mu)\}p(\mu) \\
    &=& \{\Pi_{n=1}^{N} \mathcal{N}(x_n|\mu, \lambda^{-1})\} \mathcal{N}(\mu|m,\lambda_{\mu}^{-1})
\end{eqnarray}
$$

対数をとると、

$$
\begin{eqnarray}
    p(\mu|\mathcal{D}) &=& \Sigma_{n=1}^{N} \ln \mathcal{N}(x_n|\mu,\lambda^{-1}) + \ln \mathcal{N}(\mu|m,\lambda_{\mu}^{-1}) + const. \\
    &=& \Sigma_{n=1}^{N} \{-\frac{1}{2}(x_n-\mu)^2 \lambda\} + \{-\frac{1}{2}(\mu-m)^2 \lambda_{\mu}\} + const. \\
    &=& -\frac{1}{2}\left[\Sigma_{n=1}^{N}\{(x_n^2 -2x_n\mu + \mu^2)\lambda\} + (\mu^2 - 2\mu m + m^2)\lambda_{\mu}\right] + const. \\
    &=& -\frac{1}{2}\{(N\lambda + \lambda_{\mu})\mu^2 - 2(\Sigma_{n=1}^{N} x_n \lambda + m \lambda_{\mu})\mu\} + const.
\end{eqnarray}
$$

最後の変形では$\mu$に関わらない値は$const.$に吸収させている。これを平方完成するとガウス分布の形になっていることがわかるがここでは結果から逆算的に求める。事後分布が次のようなガウス分布で書けるとする。

$$
p(\mu|\mathcal{D}) = \mathcal{N}(\mu|\hat{m},\hat{\lambda_{\mu}}^{-1})
$$

対数をとると

$$
\begin{eqnarray}
    \ln p(\mu|\mathcal{D}) &=& -\frac{1}{2}(\mu-\hat{m})^2 \hat{\lambda_{\mu}} \\
    &=& -\frac{1}{2}\{\hat{\lambda_{\mu}} \mu^2 - 2\hat{m} \hat{\lambda_{\mu}} \mu\} + const.
\end{eqnarray}
$$

となるので、事後分布のパラメータ$\hat{m},\hat{\lambda_{\mu}}$は次のように求まる。

$$
\begin{eqnarray}
    \hat{\lambda_{\mu}} &=& N \lambda + \lambda_{\mu} \\
    \hat{m} &=& \frac{\lambda \Sigma_{n=1}^{N} x_n + \lambda_{\mu}m}{\hat{\lambda_{\mu}}}
\end{eqnarray}
$$

以上で事後分布の推論が完了した。

> Note: 更新された精度パラメータは、ハイパーパラメータに$N\lambda$だけ加えたものでありデータ数$N$が大きくなる程精度が上がる、すなわち事後分布のばらつきが小さくなりかつハイパーパラメータの影響が小さくなることを示している。平均パラメータはハイパーパラメータ$m$と$\Sigma{n=1}^{N}x_n$の重み付き和の形になっておりこれもデータ数が増えるほどハイパーパラメータの影響を受けにくくなることがわかる。

## 予測分布の算出

モデル全体の事後分布をパラメータ$\mu$で周辺化することで未知の観測$x_*$に対する予測分布が得られる。ハット記号を付けるのが面倒なので、学習していない事前分布から予測分布を算出し、後で更新されたパラメータを代入してみることにする。

$$
\begin{eqnarray}
    p(x_*) &=& \int p(x_*|\mu) p(\mu) d\mu \\
    &=& \int \mathcal{N}(x_*|\mu,\lambda^{-1})\mathcal{N}(\mu|m,\lambda_{\mu}^{-1}) d\mu
\end{eqnarray}
$$

これを直接計算するのは大変なので、ベイズの定理と対数をうまく使ってみる。ベイズの定理から、

$$
p(\mu|x_*) = \frac{p(x_*|\mu)p(\mu)}{p(x_*)}
$$

$p(x_*)$を左辺に置いて対数をとると、

$$
\ln p(x_*) = \ln p(x_*|\mu) - \ln p(\mu|x_*) + const.
$$

$\ln p(\mu)$は$x_*$には関係ないので$const.$とした。$p(\mu|x_*)$は$p(\mu|\mathcal{D})$に$\mathcal{D}=x_*$とすれば求まる。

$$
\begin{eqnarray}
    p(\mu|x_*) &=& \mathcal{N}(\mu|m(x_*), (\lambda + \lambda_{\mu})^{-1}) \\
    ただし　m(x_*) &=& \frac{\lambda x_* + \lambda_{\mu}m}{\lambda + \lambda_{\mu}}
\end{eqnarray}
$$

これと$p(x_*|\mu) = \mathcal{N}(\mu, \lambda^{-1})$を代入すると


$$
\begin{eqnarray}
    \ln p(x_*) &=& \ln p(x_*|\mu) - \ln p(\mu|x_*) + const. \\
    &=& -\frac{1}{2} \{ \lambda (x_* - \mu)^2 - (\lambda + \lambda_{\mu})(\mu - m(x_*))^2 \} + const. \\
    &=& -\frac{1}{2} \{ \lambda x_*^2 - 2 \lambda \mu x_* - \frac{\lambda^2 x_*^2 + 2 \lambda \lambda_{\mu} m x_*}{\lambda + \lambda_{\mu}} + 2 \lambda \mu x_* \} + const. \\
    &=& -\frac{1}{2} \{ \frac{\lambda^2 x_*^2 + \lambda \lambda_{\mu} x_*^2 - \lambda^2 x_*^2 - 2 \lambda \lambda_{\mu} m x_*}{\lambda + \lambda_{\mu}}\} + const. \\
    &=& -\frac{1}{2} \{ \frac{\lambda \lambda_{\mu}}{\lambda + \lambda_{\mu}} x_*^2 - \frac{2m\lambda\lambda_{\mu}}{\lambda + \lambda_{\mu}} x_* \} + const.
\end{eqnarray}
$$

$x_*$の2次関数の形にできたので事後分布と同じく逆算的に計算すると、予測分布$p(x_*)$は

$$
\begin{eqnarray}
    p(x_*) &=& \mathcal{N} (x_* | \mu_*, \lambda_{*}^{-1}) \\
    ただし　\lambda_* &=& \frac{\lambda \lambda_{\mu}}{\lambda + \lambda_{\mu}} \\
    \mu_* &=& m
\end{eqnarray}
$$

となる。これに$\hat{m}, \hat{\lambda_{\mu}}$を代入することで学習した後の予測分布$p(x_*|\mathcal{D})$は

$$
\begin{eqnarray}
    p(x_*) &=& \mathcal{N} (x_* | \mu_*, \lambda_{*}^{-1}) \\
    ただし　\lambda_* &=& \frac{\lambda (N\lambda + \lambda_{\mu})}{\lambda + (N\lambda + \lambda_{\mu})} \\
    \mu_* &=& \frac{\lambda \Sigma_{n=1}^{N} x_n + \lambda_{\mu}m}{N\lambda + \lambda_{\mu}}
\end{eqnarray}
$$

と求まる。

> Note: 精度から見た結果の意味

> 精度について逆数をとると意味がわかりやすい。
>
> $$\lambda_*^{-1} = \lambda^{-1} + \hat{\lambda_{\mu}}^{-1}$$
>
> 精度の逆数は分散なので、この式は「予測分布の分散は観測分布の分散と事後分布の分散の和である」という意味になる。
>
> 今回は観測分布の分散が実際の分散に等しいことを仮定しているので、データ数が増え事後分布の分散が小さくなれば予測分布は実際の分布の分散とほぼ一致する。

# 平均が既知、精度が未知の場合

執筆中

# 平均と精度がともに未知の場合

執筆中

{{ '[『ベイズ推論による機械学習入門』(須山敦志、講談社)](https://www.kspub.co.jp/book/detail/1538320.html)' | fndetail: 1 }}